<a href="https://colab.research.google.com/github/chineidu/NLP-Tutorial/blob/main/notebook/06_Transformers/6b_types_of_tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Byte-Pair Encoding Tokenization (BPE)](https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt)

```text
- Byte-Pair Encoding (BPE) was initially developed as an algorithm to compress texts, and then used by OpenAI for tokenization when pretraining the GPT model.
- It’s used by a lot of Transformer models, including GPT, GPT-2, RoBERTa, BART, and DeBERTa.
```

In [1]:
!pip install rich
!pip install transformers[torch]
!pip install torch datasets evaluate

In [2]:
# Built-in library
import re
import json
from typing import Any, Dict, List, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import pandas as pd
from rich import print

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
# %load_ext lab_black

# auto reload imports
# %load_ext autoreload
# %autoreload 2

### Training algorithm

```text
- Byte-Pair Encoding Tokenization (BPE) training starts by computing the unique set of words used in the corpus (after the normalization and pre-tokenization steps are completed), then building the vocabulary by taking all the symbols used to write those words.
- As a very simple example, let’s say our corpus uses these five words:
  "hug", "pug", "pun", "bun", "hugs"

- The base vocabulary will then be ["b", "g", "h", "n", "p", "s", "u"].
- For real-world cases, that base vocabulary will contain all the ASCII characters, at the very least, and probably some Unicode characters as well.
- If an example you are tokenizing uses a character that is not in the training corpus, that character will be converted to the unknown token.
- That’s one reason why lots of NLP models are very bad at analyzing content with emojis, for instance.

- The GPT-2 and RoBERTa tokenizers (which are pretty similar) have a clever way to deal with this: they don’t look at words as being written with Unicode characters, but with bytes.
- This way the base vocabulary has a small size (256), but every character you can think of will still be included and not end up being converted to the unknown token.
- This trick is called byte-level BPE.
- After getting this base vocabulary, we add new tokens until the desired vocabulary size is reached by learning merges, which are rules to merge two elements of the existing vocabulary together into a new one.
- So, at the beginning these merges will create tokens with two characters, and then, as training progresses, longer subwords.
- At any step during the tokenizer training, the BPE algorithm will search for the most frequent pair of existing tokens (by “pair,” here we mean two consecutive tokens in a word).
- That most frequent pair is the one that will be merged, and we rinse and repeat for the next step.

- Going back to our previous example, let’s assume the words had the following frequencies:
```

```python
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
```

```text
meaning "hug" was present 10 times in the corpus, "pug" 5 times, "pun" 12 times, "bun" 4 times, and "hugs" 5 times. We start the training by splitting each word into characters (the ones that form our initial vocabulary) so we can see each word as a list of tokens:
```

```python
("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)
```

```text
Then we look at pairs. The pair ("h", "u") is present in the words "hug" and "hugs", so 15 times total in the corpus. It’s not the most frequent pair, though: that honor belongs to ("u", "g"), which is present in "hug", "pug", and "hugs", for a grand total of 20 times in the vocabulary.

Thus, the first merge rule learned by the tokenizer is ("u", "g") -> "ug", which means that "ug" will be added to the vocabulary, and the pair should be merged in all the words of the corpus. At the end of this stage, the vocabulary and corpus look like this:
```

```python
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug"]
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)
```

<br>

```text
- Now we have some pairs that result in a token longer than two characters: the pair ("h", "ug"), for instance (present 15 times in the corpus). The most frequent pair at this stage is ("u", "n"), however, present 16 times in the corpus, so the second merge rule learned is ("u", "n") -> "un".
- Adding that to the vocabulary and merging all existing occurrences leads us to:
```

```python
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un"]
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("h" "ug" "s", 5)
```

```text
Now the most frequent pair is ("h", "ug"), so we learn the merge rule ("h", "ug") -> "hug", which gives us our first three-letter token. After the merge, the corpus looks like this:
```

```python
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]
Corpus: ("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)
```

<hr><br>

### Tokenization algorithm
Tokenization follows the training process closely, in the sense that new inputs are tokenized by applying the following steps:

```text
1. Normalization
2. Pre-tokenization
3. Splitting the words into individual characters
4. Applying the merge rules learned in order on those splits

Let’s take the example we used during training, with the three merge rules learned:
```

```python

("u", "g") -> "ug"
("u", "n") -> "un"
("h", "ug") -> "hug"
```

```text
- The word "bug" will be tokenized as ["b", "ug"]. "mug", however, will be tokenized as ["[UNK]", "ug"] since the letter "m" was not in the base vocabulary.
- Likewise, the word "thug" will be tokenized as ["[UNK]", "hug"]: the letter "t" is not in the base vocabulary, and applying the merge rules results first in "u" and "g" being merged and then "hu" and "g" being merged.
```

<br><br>

#### Implementing Byte-Pair Encoding Tokenization (BPE)

```text
- Now let’s take a look at an implementation of the BPE algorithm.
- This won’t be an optimized version you can actually use on a big corpus; we just want to show you the code so you can understand the algorithm a little bit better.
- First we need a corpus, so let’s create a simple one with a few sentences:
```

In [3]:
from transformers import AutoTokenizer


tokenizer:AutoTokenizer = AutoTokenizer.from_pretrained("gpt2")

# Pre-tokenize the corpus into words. Since we are replicating a BPE tokenizer (like GPT-2), we will use
# the gpt2 tokenizer for the pre-tokenization:
corpus: list[str] = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [4]:
from collections import defaultdict


# Compute the frequencies of each word in the corpus as we do the pre-tokenization:
word_freqs: defaultdict = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, _ in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

defaultdict(<class 'int'>, {
    'This': 3,
    'Ġis': 2,
    'Ġthe': 1,
    'ĠHugging': 1,
    'ĠFace': 1,
    'ĠCourse': 1,
    '.': 4,
    'Ġchapter': 1,
    'Ġabout': 1,
    'Ġtokenization': 1,
    'Ġsection': 1,
    'Ġshows': 1,
    'Ġseveral': 1,
    'Ġtokenizer': 1,
    'Ġalgorithms': 1,
    'Hopefully': 1,
    ',': 1,
    'Ġyou': 1,
    'Ġwill': 1,
    'Ġbe': 1,
    'Ġable': 1,
    'Ġto': 1,
    'Ġunderstand': 1,
    'Ġhow': 1,
    'Ġthey': 1,
    'Ġare': 1,
    'Ġtrained': 1,
    'Ġand': 1,
    'Ġgenerate': 1,
    'Ġtokens': 1
})

In [5]:
type(word_freqs)

collections.defaultdict

In [6]:
# Compute the base vocabulary, formed by all the characters used in the corpus:
alphabet: list[str] = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

[
    ',',
    '.',
    'C',
    'F',
    'H',
    'T',
    'a',
    'b',
    'c',
    'd',
    'e',
    'f',
    'g',
    'h',
    'i',
    'k',
    'l',
    'm',
    'n',
    'o',
    'p',
    'r',
    's',
    't',
    'u',
    'v',
    'w',
    'y',
    'z',
    'Ġ'
]

In [7]:
# Add the special tokens used by the model at the beginning of that vocabulary. In the case of GPT-2,
# the only special token is "<|endoftext|>":

vocab: list[str] = ["<|endoftext|>"] + alphabet.copy()

# Split each word into individual characters, to be able to start training:
splits: dict[str, Any] = {word: [c for c in word] for word in word_freqs.keys()}

In [8]:
splits

{'This': ['T', 'h', 'i', 's'],
 'Ġis': ['Ġ', 'i', 's'],
 'Ġthe': ['Ġ', 't', 'h', 'e'],
 'ĠHugging': ['Ġ', 'H', 'u', 'g', 'g', 'i', 'n', 'g'],
 'ĠFace': ['Ġ', 'F', 'a', 'c', 'e'],
 'ĠCourse': ['Ġ', 'C', 'o', 'u', 'r', 's', 'e'],
 '.': ['.'],
 'Ġchapter': ['Ġ', 'c', 'h', 'a', 'p', 't', 'e', 'r'],
 'Ġabout': ['Ġ', 'a', 'b', 'o', 'u', 't'],
 'Ġtokenization': ['Ġ',
  't',
  'o',
  'k',
  'e',
  'n',
  'i',
  'z',
  'a',
  't',
  'i',
  'o',
  'n'],
 'Ġsection': ['Ġ', 's', 'e', 'c', 't', 'i', 'o', 'n'],
 'Ġshows': ['Ġ', 's', 'h', 'o', 'w', 's'],
 'Ġseveral': ['Ġ', 's', 'e', 'v', 'e', 'r', 'a', 'l'],
 'Ġtokenizer': ['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'],
 'Ġalgorithms': ['Ġ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's'],
 'Hopefully': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y'],
 ',': [','],
 'Ġyou': ['Ġ', 'y', 'o', 'u'],
 'Ġwill': ['Ġ', 'w', 'i', 'l', 'l'],
 'Ġbe': ['Ġ', 'b', 'e'],
 'Ġable': ['Ġ', 'a', 'b', 'l', 'e'],
 'Ġto': ['Ġ', 't', 'o'],
 'Ġunderstand': ['Ġ', 'u', 'n'

In [9]:
def compute_pair_freqs(splits) -> defaultdict:
    pair_freqs: defaultdict = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [10]:
# Let’s have a look at a part of this dictionary after the initial splits:
pair_freqs: defaultdict = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3

('h', 'i'): 3

('i', 's'): 5

('Ġ', 'i'): 2

('Ġ', 't'): 7

('t', 'h'): 3

In [11]:
# Now, finding the most frequent pair only takes a quick loop:
best_pair: str = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't')
7

In [12]:
# So the first merge to learn is ('Ġ', 't') -> 'Ġt', and we add 'Ġt' to the vocabulary:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

In [13]:
# To continue, we need to apply that merge in our splits dictionary. Let’s write another function for this:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [14]:
# And we can have a look at the result of the first merge:
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

['Ġt', 'r', 'a', 'i', 'n', 'e', 'd']

In [15]:

vocab_size: int = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [16]:
# As a result, we’ve learned 19 merge rules (the initial vocabulary had a size of 31 — 30 characters in
# the alphabet, plus the special token):
print(merges)

{
    ('Ġ', 't'): 'Ġt',
    ('i', 's'): 'is',
    ('e', 'r'): 'er',
    ('Ġ', 'a'): 'Ġa',
    ('Ġt', 'o'): 'Ġto',
    ('e', 'n'): 'en',
    ('T', 'h'): 'Th',
    ('Th', 'is'): 'This',
    ('o', 'u'): 'ou',
    ('s', 'e'): 'se',
    ('Ġto', 'k'): 'Ġtok',
    ('Ġtok', 'en'): 'Ġtoken',
    ('n', 'd'): 'nd',
    ('Ġ', 'is'): 'Ġis',
    ('Ġt', 'h'): 'Ġth',
    ('Ġth', 'e'): 'Ġthe',
    ('i', 'n'): 'in',
    ('Ġa', 'b'): 'Ġab',
    ('Ġtoken', 'i'): 'Ġtokeni'
}

In [17]:
print(vocab)

[
    '<|endoftext|>',
    ',',
    '.',
    'C',
    'F',
    'H',
    'T',
    'a',
    'b',
    'c',
    'd',
    'e',
    'f',
    'g',
    'h',
    'i',
    'k',
    'l',
    'm',
    'n',
    'o',
    'p',
    'r',
    's',
    't',
    'u',
    'v',
    'w',
    'y',
    'z',
    'Ġ',
    'Ġt',
    'is',
    'er',
    'Ġa',
    'Ġto',
    'en',
    'Th',
    'This',
    'ou',
    'se',
    'Ġtok',
    'Ġtoken',
    'nd',
    'Ġis',
    'Ġth',
    'Ġthe',
    'in',
    'Ġab',
    'Ġtokeni'
]

In [18]:
# To tokenize a new text, we pre-tokenize it, split it, then apply all the merge rules learned:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

In [19]:
# We can try this on any text composed of characters in the alphabet:
tokenize("This is not a token.")

['This', 'Ġis', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']

## [WordPiece Tokenization](https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt)

```text
- WordPiece is the tokenization algorithm Google developed to pretrain BERT.
- It has since been reused in quite a few Transformer models based on BERT, such as DistilBERT, MobileBERT, Funnel Transformers, and MPNET.
- It’s very similar to BPE in terms of the training, but the actual tokenization is done differently.
```

### Training algorithm

```text
⚠️ Google never open-sourced its implementation of the training algorithm of WordPiece, so what follows is our best guess based on the published literature. It may not be 100% accurate.

Like BPE, WordPiece starts from a small vocabulary including the special tokens used by the model and the initial alphabet.
Since it identifies subwords by adding a prefix (like ## for BERT), each word is initially split by adding that prefix to all the characters inside the word. So, for instance, "word" gets split like this:
```

```python
w ##o ##r ##d
```

```text
Thus, the initial alphabet contains all the characters present at the beginning of a word and the characters present inside a word preceded by the WordPiece prefix.

Let’s look at the same vocabulary we used in the BPE training example:
```

```python
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)

# The splits here will be:

("h" "##u" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("h" "##u" "##g" "##s", 5)
```

```text
so the initial vocabulary will be ["b", "h", "p", "##g", "##n", "##s", "##u"] (if we forget about special tokens for now). The most frequent pair is ("##u", "##g") (present 20 times), but the individual frequency of "##u" is very high, so its score is not the highest (it’s 1 / 36). All pairs with a "##u" actually have that same score (1 / 36), so the best score goes to the pair ("##g", "##s") — the only one without a "##u" — at 1 / 20, and the first merge learned is ("##g", "##s") -> ("##gs").

Note that when we merge, we remove the ## between the two tokens, so we add "##gs" to the vocabulary and apply the merge in the words of the corpus:
```

```python
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs"]
Corpus: ("h" "##u" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("h" "##u" "##gs", 5)
```

```text
At this point, "##u" is in all the possible pairs, so they all end up with the same score. Let’s say that in this case, the first pair is merged, so ("h", "##u") -> "hu". This takes us to:
```

```python
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs", "hu"]
Corpus: ("hu" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("hu" "##gs", 5)
```

```text
Then the next best score is shared by ("hu", "##g") and ("hu", "##gs") (with 1/15, compared to 1/21 for all the other pairs), so the first pair with the biggest score is merged:
```

```python
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs", "hu", "hug"]
Corpus: ("hug", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("hu" "##gs", 5)

```

### Tokenization Algorithm

```
- Tokenization differs in WordPiece and BPE in that WordPiece only saves the final vocabulary, not the merge rules learned.
- Starting from the word to tokenize, WordPiece finds the longest subword that is in the vocabulary, then splits on it.
- For instance, if we use the vocabulary learned in the example above, for the word "hugs" the longest subword starting from the beginning that is inside the vocabulary is "hug", so we split there and get ["hug", "##s"].
- We then continue with "##s", which is in the vocabulary, so the tokenization of "hugs" is ["hug", "##s"].

- With BPE, we would have applied the merges learned in order and tokenized this as ["hu", "##gs"], so the encoding is different.

- As another example, let’s see how the word "bugs" would be tokenized. "b" is the longest subword starting at the beginning of the word that is in the vocabulary, so we split there and get ["b", "##ugs"].
- Then "##u" is the longest subword starting at the beginning of "##ugs" that is in the vocabulary, so we split there and get ["b", "##u, "##gs"].
- Finally, "##gs" is in the vocabulary, so this last list is the tokenization of "bugs".

- When the tokenization gets to a stage where it’s not possible to find a subword in the vocabulary, the whole word is tokenized as unknown — so, for instance, "mug" would be tokenized as ["[UNK]"], as would "bum" (even if we can begin with "b" and "##u", "##m" is not the vocabulary, and the resulting tokenization will just be ["[UNK]"], not ["b", "##u", "[UNK]"]).
- This is another difference from BPE, which would only classify the individual characters not in the vocabulary as unknown.
```

<br>

### Implementing WordPiece

```text
- Now let’s take a look at an implementation of the WordPiece algorithm. Like with BPE, this is just pedagogical, and you won’t able to use this on a big corpus.

- We will use the same corpus as in the BPE example:
```